## Загрузить или продолжить работу

In [ ]:
import pandas as pd
df = pd.read_pickle("filename")
df.shape

In [3]:
# примерный DF
df.head()

,screen_name,date_time,location,text
0,leila_fong,2019-10-23 23:57:59,"Illinois, USA","@ATTHelp I’m blocking, but then they change th..."
1,SnackEvangelist,2019-10-23 23:57:44,,@ATTHelp Just logged into my account and it’s ...
2,DeathboxTy,2019-10-23 23:54:34,,@ATTHelp can’t get my Visual Voicemail to work...
3,BLoLovesYou,2019-10-23 23:54:05,"San Antonio, TX",@ATTHelp You were charging me $150 for an outd...
4,ATTHelp,2019-10-23 23:53:02,,"@leila_fong Hey, Leila, we're here to help. Cl..."


## Pre-processing tweets

Провести обработку текста:
- удалить стоп-слова 0
- найти обращения по @ или хэштеги
- найти ссылки на сайты
- разбить по словам и n-граммы

In [4]:
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import time

start_time = time.time() 

tokenizer = TweetTokenizer()
tokenized_tweets = [tokenizer.tokenize(t) for t in df.text]
stop_words = stopwords.words("english")
stop_words.extend(["&amp;", "&gt;", "&lt;"])
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

clean_text = []
handles = []
hashtags = []
carrier = []

# простой пример pre-processing
for tweet in tokenized_tweets:
    thandles = []
    thashtags = []
    joined_words = " "
    words = [w.lower() for w in tweet if len(w)>2 and w not in stop_words]
    thandles = [w for w in words if re.search("^@\w+", w)]
    tlinks = [w for w in words if re.search("^https:\w+", w)]
    thashtags = [w for w in words if re.search("^#\w+", w)]
    words = [w for w in words if w.isalpha() and w not in thandles and w not in thashtags and w not in tlinks]
    words = [lemmatizer.lemmatize(w) for w in words]
    
    joined_words = joined_words.join(words)
    tlength = len(joined_words)
    handles.append(thandles)
    hashtags.append(thashtags)
    clean_text.append(joined_words)
    

Elapsed clock time:  0.008540990286403233  seconds


In [5]:
# собрать надо что-такое
df.head()

,screen_name,date_time,location,text,clean_text,handles,hashtags
0,leila_fong,2019-10-23 23:57:59,"Illinois, USA","@ATTHelp I’m blocking, but then they change th...",blocking change number one digit switch unknown,[@atthelp],[]
1,SnackEvangelist,2019-10-23 23:57:44,,@ATTHelp Just logged into my account and it’s ...,just logged account added how get rid,[@atthelp],[]
2,DeathboxTy,2019-10-23 23:54:34,,@ATTHelp can’t get my Visual Voicemail to work...,get visual voicemail work suggestion tried tip...,[@atthelp],[]
3,BLoLovesYou,2019-10-23 23:54:05,"San Antonio, TX",@ATTHelp You were charging me $150 for an outd...,you charging outdated unlimited plan switched ...,[@atthelp],[]
4,ATTHelp,2019-10-23 23:53:02,,"@leila_fong Hey, Leila, we're here to help. Cl...",hey leila help click learn protect unwanted call,[@leila_fong],[]


### Разработка сегментов

In [6]:
# изучить textblob
# https://textblob.readthedocs.io/en/dev/

from textblob import TextBlob
polarity = [0.0]*len(df)
subjectivity = [0.5]*len(df)
sentiment = [""]*len(df)

for i in df.index:
    blob = TextBlob(df.clean_text[i])
    polarity[i] = blob.polarity
    subjectivity[i] = blob.subjectivity
    
    # разбивки на сегменты 
    # а какие способы ты знаешь ещё?
    if polarity[i] > 0.0:
        sentiment[i] = "pos"
    elif polarity[i] < 0.0:
        sentiment[i] = "neg"
    else:
        sentiment[i] = "neu"

df["sentiment"]    = sentiment
df["polarity"]     = polarity
df["subjectivity"] = subjectivity

### Сделаем разбивку по операторам

In [7]:
# создадим колонку, которая укажет, какой это оператор

carrier = []

for text in df[['screen_name','text']].values:
    screen_name = text[0].lower()
    twitter_text = text[1].lower()
    current_carrier = []
    if 'имя оператора' in twitter_text or 'имя оператора' in screen_name:
        current_carrier.append('имя оператора')
    
    ... 
    carrier.append(' '.join(current_carrier))

df['carrier'] = carrier
        

In [ ]:
# должен включать в себя такие колонки 100%
df[['carrier','location','date_time','clean_text','sentiment']].head()

In [9]:
# проверь, чтобы не было пустых, то есть без оператора!

df = df[df.carrier != '']
df.shape

(171423, 11)

In [12]:
# сделай какой-нибудь токинайзер
# https://ru.wikipedia.org/wiki/%D0%9B%D0%B5%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7
# https://docs.python.org/3/library/tokenize.html

tokenized_tweets = [tokenizer.tokenize(t) for t in df.text]
clf_text = []
for tweet in tokenized_tweets:
    joined_words = " "
    words = [w.lower() for w in tweet if w.isalpha() or w[0]=='@']
    tlinks = [w for w in words if re.search("^https:\w+", w)]
    words = [w for w in words if  w not in tlinks]
    joined_words = joined_words.join(words)
    clf_text.append(joined_words)

df["clf_text"] = clf_text

## Сделай обзор по группам (EDA)

Примеры:

https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python

https://www.coursera.org/projects/exploratory-data-analysis-python-pandas

https://www.analyticsvidhya.com/blog/2020/08/exploratory-data-analysiseda-from-scratch-in-python/

## Определи, какие слова наиболее характерны для операторов

- какие cлова чаще, чем остальные встречаются для определенного оператора
- посмотреть распределения по словам, относительно операторов

- пропобуй найти слова: помогите, бесплатно, обещали, покупатель, очень медленно 
- добавь свои слова ( к вышепредставленным) и сделай топ-20

## Определи время и как время сообщения распределено по времени

- определи месяц, день недели, день месяца, время дня, часть дня (до обеда, день, вечер - например)

Варианты кодирования фичей для алгоритмов (когда надо будет в модель):

https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/

https://towardsdatascience.com/feature-engineering-time-3934038e0dbe

## Сделай простой сентиментный анализ

- анализ склонности и отвращения

примеры на Python:

https://github.com/subro608/review_polarity

https://github.com/shoreason/yelp-review-polarity

https://github.com/KarthikGotham/PolarityAnalyzer

https://github.com/Ashwani018/Amazon-Fine-Food-Reviews

###  Сделай распределения и обзор по сентиментному анализу относительно операторов

In [33]:
### Попробуй найти и посмотреть распределение негативные - позитивные распределения

# пример

,carrier,negative,positive,percentage_negative,percentage_positive
0,ATT,13167,24642,0.348250,0.651750
1,Verizon,6620,19717,0.251357,0.748643
2,Tmobile,3353,14547,0.187318,0.812682


## Создание репрезентативного словаря для негативны и позитивных твитов


Для создания такого словаря нужно использовать методы:

- методы по поиску частоты встречания слова в текстах (словарь частоты)
- метод TF-IDF
- метод topic-modeling

In [35]:
# Пример словаря - частот

[('help', 2918), ('thanks', 2431), ('live', 2301), ('first', 2273), ('love', 2177)]
[('free', 4006), ('disney', 3376), ('customer', 3167), ('year', 2799), ('new', 2238)]
[('love', 2157), ('thanks', 1306), ('new', 1271), ('want', 961), ('best', 867)]


### Method2: Get representative words by Topic Modeling

In [109]:
# для метода Topic Modeling понадобятся библиотеки
from gensim.corpora import Dictionary
from gensim.models import LdaModel, LdaMulticore
from gensim.models import TfidfModel
import nltk

In [111]:
# пример вывод по методу Topic Modeling

att - pos
Topic 1 :  day, customer, reply, thing, time, wifi, year, work, att, service
Topic 2 :  josh, att, career, total, attempt, thank, part, call, chance, post
Topic 3 :  thanks, look, help, bill, phone, service, please, account, internet, people
Topic 4 :  love, today, week, ticket, month, right, hand, fee, man, sunday
Topic 5 :  episode, mini, series, offseason, court, star, player, lol, interview, http
----------
att - neg
Topic 1 :  cause, tweet, concert, celebrate, tomorro, city, idea, life, towards, att
Topic 2 :  game, help, everyone, employee, acc, vote, account, please, get, see
Topic 3 :  people, fuck, game, att, trump, mean, look, company, news, night
Topic 4 :  service, att, time, day, phone, hate, others, hour, internet, customer
Topic 5 :  board, member, stephenson, sent, thousand, samuel, email, customer, ticket, shit
----------
verizon - pos
Topic 1 :  phone, note, service, issue, area, thank, verizon, network, cable, data
Topic 2 :  thanks, work, bill, day, reward